In [1]:
import pandas as pd
import pandas_dedupe

from connectors.postgres_connector import PostgresConnector
from feeds.postgres_feed import PostgresFeed


pg_conn = PostgresConnector(db='jameycdb', username='jameyc', password='UXZSXXXSFZeU8XKw', host='silobuster-db-do-user-12298230-0.b.db.ondigitalocean.com', port=25060)
# select_qry = "SELECT id, name, address FROM what_location WHERE address is not null and not address = '' and name is not null and not name = ''"
# select_qry2 = "SELECT id, name, address FROM what_location"

select_qry = "SELECT id, name as company_name, address_1, address_2, city, state_province, postal_code, url, description, duplicate_id, duplicate_type FROM organizations_normalized WHERE name IS NOT NULL AND NOT name = '' AND address_1 IS NOT NULL AND NOT address_1 = '' AND city IS NOT NULL AND NOT city = ''"
raw_def = [
    {"field": "company_name", "type": "String"},
    {"field": "address_1", "type": "String", "has missing": True},
    {"field": "address_2", "type": "Exists", "has missing": True},
    {"field": "city", "type": "String", "has missing": True},
    {"field": "state_province", "type": "String", "has missing": True},
    {"field": "postal_code", "type": "Exists", "has missing": True},
    {"field": "url", "type": "Exists", "has missing": True}
]


pg_feed = PostgresFeed.from_manual(connector=pg_conn, query=select_qry, column_definition=raw_def, primary_key='id')

print (pg_feed.df.head(5))

df_final = pandas_dedupe.dedupe_dataframe(pg_feed.df, [
    ('company_name', 'String'),
    ('address_1', 'String'), 
    ('address_2','String', 'has missing'), 
    ('city','String','has missing'), 
    ('state_province','String','has missing'), 
    ('postal_code','String','has missing'), 
    ('url','String','has missing'), 
    ('description','String','has missing')
])

df_final.to_csv('deduplication_output.csv')

                                     id  \
0  850f83f0-8ad7-4d1e-81c0-c7b19e5d380c   
1  db0ab21f-64d9-469a-a25f-33e8765716f1   
2  9febd2f6-0a23-4a3e-979c-34add947efe9   
3  9ccfe539-c0b7-48c9-83e0-f19a630a1916   
4  d358eaa9-bee0-48b6-b61e-ee8e68657950   

                                        company_name              address_1  \
0  depressive and bipolar support alliance whatco...      800 chestnut str.   
1           bellingham parks and recreation division      210 lottie street   
2                        mount baker school district       4956 deming road   
3                        mount baker school district         4956 deming rd   
4                       bellingham technical college  3028 lindbergh avenue   

  address_2        city state_province postal_code  \
0  suite 1c  bellingham             wa       98225   
1            bellingham             wa       98225   
2                deming             wa       98244   
3                deming             wa       98244

/tmp/ipykernel_30123/1591311375.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df_final = pandas_dedupe.dedupe_dataframe(pg_feed.df, [


# duplicate sets 372
